In [65]:
import os
import sys
import pandas as pd
from sklearn.linear_model import LogisticRegression
import pickle
sys.path.append(os.path.abspath('../Scripts'))
import warnings
warnings.filterwarnings('ignore')

In [66]:
import eda as ed

In [67]:
import Model as m

In [68]:
df = 'features_binned.csv'
features = ed.load_data(df)

2024-10-12 08:23:29,750 - INFO - Attempting to load data from features_binned.csv
2024-10-12 08:23:29,750 - INFO - Attempting to load data from features_binned.csv
2024-10-12 08:23:29,800 - INFO - Successfully loaded data from features_binned.csv
2024-10-12 08:23:29,800 - INFO - Successfully loaded data from features_binned.csv


In [69]:
features

,CustomerId,Transaction_Frequency,Avg_Transaction_Amount,Total_Transaction_Volume,Transaction_Recency,Transaction_Amount_Variability,Avg_Transaction_Hour,Avg_Transaction_Day,Avg_Transaction_Month,Avg_Transaction_Year,RFMS_Score,Risk_Label,Recency_Bin,Frequency_Bin,Volume_Bin,Recency_WoE
0,CustomerId_1,0.000000,0.045974,0.556885,0.923077,0.005076,0.695652,0.666667,0.909091,0.000000,0.369991,Good,Very Old,Very Low,Very Low,1.000000
1,CustomerId_10,0.000000,0.045974,0.556885,0.923077,0.005076,0.695652,0.666667,0.909091,0.000000,0.369991,Good,Very Old,Very Low,Very Low,1.000000
2,CustomerId_1001,0.000978,0.047525,0.557044,0.978022,0.001982,0.339130,0.500000,0.909091,0.000000,0.384011,Good,Very Old,Low,Low,1.000000
3,CustomerId_1002,0.002445,0.047124,0.556961,0.285714,0.000169,0.584980,0.457576,0.694215,0.272727,0.211280,Good,Old,High,Low,1.000000
4,CustomerId_1003,0.001222,0.047451,0.557044,0.131868,0.001822,0.623188,0.000000,0.090909,1.000000,0.172534,Bad,Recent,Low,Low,0.028200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,CustomerId_992,0.001222,0.047451,0.557044,0.054945,0.001839,0.434783,0.233333,0.090909,1.000000,0.153303,Bad,Very Recent,Low,Low,0.002047
3738,CustomerId_993,0.000978,0.047525,0.557044,0.285714,0.002038,0.652174,0.566667,0.000000,1.000000,0.210934,Good,Old,Low,Low,1.000000
3739,CustomerId_994,0.024450,0.047678,0.559826,0.010989,0.004472,0.604391,0.582508,0.382538,0.623762,0.148816,Bad,Very Recent,Very High,Very High,0.002047
3740,CustomerId_996,0.003912,0.047988,0.557676,0.747253,0.001339,0.629156,0.200000,1.000000,0.000000,0.327210,Good,Very Old,High,Very High,1.000000


In [70]:
m.logistic_regression_model_evaluation(features)

2024-10-12 08:23:30,088 - INFO - logistic regresion Model_evaluation.....
2024-10-12 08:23:30,088 - INFO - logistic regresion Model_evaluation.....
Accuracy: 99.07%
Precision: 1.00
Recall: 0.98
F1 Score: 0.99
ROC-AUC: 1.00
Confusion Matrix:
[[378   0]
 [  7 364]]


In [71]:
m.random_forest_model_evaluation(features)

2024-10-12 08:23:30,242 - INFO - random forest Model_evaluation.....
2024-10-12 08:23:30,242 - INFO - random forest Model_evaluation.....
Accuracy: 0.9879839786381842
Precision: 0.994535519125683
Recall: 0.9811320754716981
F1 Score: 0.9877883310719131
ROC-AUC: 0.9939852251173006
Confusion Matrix: [[376   2]
 [  7 364]]


The logistic regression model evaluation results based on the binned features are as follows:

    Accuracy: 99.07% – The model correctly predicts 99.07% of the total observations.
    Precision: 100% – All positive predictions (classified as "Good") are correct.
    Recall (Sensitivity): 98% – The model identifies 98% of the actual "Good" cases.
    F1 Score: 99% – A balanced measure between precision and recall.
    ROC-AUC: 100% – The model's ability to distinguish between "Good" and "Bad" classes is excellent.

so compared to the random forest we got logestic_regression the best although the results are near

so we are going to fit the logistic regression Model

In [72]:
# Extract the relevant features for prediction
feature_names = ['Risk_Label', 'Recency_WoE', 'Transaction_Frequency', 'Total_Transaction_Volume']

# Ensure the dataset contains only the required features
features = features[feature_names]
print(features.head())


  Risk_Label  Recency_WoE  Transaction_Frequency  Total_Transaction_Volume
0       Good       1.0000               0.000000                  0.556885
1       Good       1.0000               0.000000                  0.556885
2       Good       1.0000               0.000978                  0.557044
3       Good       1.0000               0.002445                  0.556961
4        Bad       0.0282               0.001222                  0.557044


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [74]:
# Use only WoE transformed features and Risk_Label for modeling
X = features[['Recency_WoE', 'Transaction_Frequency', 'Total_Transaction_Volume']]
y = features['Risk_Label'].map({'Good': 1, 'Bad': 0})  # Encode labels as 1 for Good, 0 for Bad

    # Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

    # Make predictions
y_pred = logistic_model.predict(X_test)

In [75]:
m.fitting_model

<function Model.fitting_model(X, y)>

In [76]:
final_model = LogisticRegression(random_state=42)
final_model.fit(X, y)

LogisticRegression(random_state=42)

In [77]:

# Save the model
with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(final_model, model_file)


In [91]:
# Load the model
with open('logistic_regression_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Predicting on new data (replace `new_data` with actual new data)
new_data_predictions = loaded_model.predict(new_data)


In [92]:
new_data

,Recency_WoE,Transaction_Frequency,Total_Transaction_Volume
3450,0.002047,0.009535,0.557658
1114,0.002047,0.000244,0.557103
351,0.002047,0.000733,0.556991
1983,0.002047,0.010513,0.557213
2321,0.028200,0.000000,0.556965


In [93]:
features

,Risk_Label,Recency_WoE,Transaction_Frequency,Total_Transaction_Volume
0,Good,1.000000,0.000000,0.556885
1,Good,1.000000,0.000000,0.556885
2,Good,1.000000,0.000978,0.557044
3,Good,1.000000,0.002445,0.556961
4,Bad,0.028200,0.001222,0.557044
...,...,...,...,...
3737,Bad,0.002047,0.001222,0.557044
3738,Good,1.000000,0.000978,0.557044
3739,Bad,0.002047,0.024450,0.559826
3740,Good,1.000000,0.003912,0.557676


In [95]:
# Print feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': final_model.coef_[0]
})

print(feature_importance.sort_values(by='Coefficient', ascending=False))


                    Feature  Coefficient
0               Recency_WoE     9.385466
1     Transaction_Frequency     0.958783
2  Total_Transaction_Volume     0.948137
